# Random Forests

## https://colab.research.google.com/
## https://github.com/focods/ds_random_forest

<img src="https://raw.githubusercontent.com/FevetS/ds_random_forest/master/media/bobross_dt.PNG"  style="width: 50%; height: 50%">

# Why Random Forests?
### 1. Easy - little or no tuning, handles missing values, no need to normalize variables
### 2. Versatile - classification, regression, binary probability, variable selection
### 3. Decent Accuracy - often as good as more complex and harder to use models

# What are Random Forests?
## Just a bunch of trees

# Decision Trees

Everyone knows decision trees. They're just binary flow charts.

<img src="https://s3.amazonaws.com/theoatmeal-img/comics/selfie_stick/selfie_stick.jpg" style="width: 50%; height: 50%">

Decision tree form:

In [ ]:
import graphviz
dot = graphviz.Digraph(comment='Should you buy a selfie stick?')
dot.node('A', 'given up on life?')
dot.node('B', 'YES:\nbuy a selfie stick')
dot.node('J', 'NO:\nJapanese?')
dot.node('C', 'YES:\nbuy a selfie stick')
dot.node('D', "NO:\ndon't buy a selfie stick")

dot.edges(['AB', 'AJ', 'JC', 'JD'])

dot

## How are splits made in classification?
**Gini Impurity**  
The probability of an incorrect classification for a new sample given the distribution of samples in the set.

\begin{align}\operatorname{I}_{G}(p) = \sum_{i=1}^{J} p_i (1-p_i) 
\end{align}

where, $p_i$ is the fraction of samples with class i from J classes. 

* Gini has a minimum of 0 when all samples are in a single class.  
* It approaches a maximum of 1 as samples are evenly distributed among an infinite number of classes.

In a decision tree, splitting at a node is determined by the criteria that causes the greatest decrease in Gini from the set of samples before the split to the weighted average of Gini of both branches after the split.

Entropy (or Information Gain) can be used instead of Gini but is more computationaly expensive and produces nearly the same result.

## Quit a few parameters for tuning the tree
1. Maximum depth
2. Minimum number of samples to split at a node
3. Minimum number of samples in a leaf
4. Maximum number of features to consider at a node

## A basic example with the iris data

In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.model_selection import train_test_split

# load data set and train/test split
iris = load_iris()
X = iris.data[:, :2]
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42)

In [ ]:
# A complex decision tree with default settings
def_clf = tree.DecisionTreeClassifier()
def_clf = def_clf.fit(X_train, y_train)

def_clf.get_params()

In [ ]:
# A simple decision tree with max_depth=2
simp_clf = tree.DecisionTreeClassifier(max_depth=2)
simp_clf = simp_clf.fit(X_train, y_train)

**What do the trees look like?**

In [ ]:
# # Shrink svg to make them more visible
# from IPython.display import HTML
# style = "<style>svg{width:20% !important;height:20% !important;}</style>"
# HTML(style);

In [ ]:
# The default tree
dot_data = tree.export_graphviz(def_clf, out_file=None, 
                     feature_names=iris.feature_names[:2],  
                     class_names=iris.target_names,  
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)
graph

In [ ]:
# The simple tree
dot_data = tree.export_graphviz(simp_clf, out_file=None, 
                     feature_names=iris.feature_names[:2],  
                     class_names=iris.target_names,  
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)
graph

**What's the effect of a complex tree on prediction?**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def decision_surface(ax, clf, X, y, title):
    axx, axy = X[:, 0], X[:, 1]
    axx_min, axx_max = axx.min() - 1, axx.max() + 1
    axy_min, axy_max = axy.min() - 1, axy.max() + 1
    xx, yy = np.meshgrid(np.arange(axx_min, axx_max, 0.02),
                         np.arange(axy_min, axy_max, 0.02))
    
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    ax.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
    ax.scatter(axx, axy, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
    ax.set(xlim=[xx.min(), xx.max()], ylim=[yy.min(), yy.max()],
           xlabel='Sepal length', ylabel='Sepal width', title=title)
    return ax

fig, axs = plt.subplots(2, 2, figsize = (8,8))
decision_surface(axs.flat[0], def_clf, X_train, y_train, title='Default - Training')
decision_surface(axs.flat[1], def_clf, X_test, y_test, title='Default - Test')
decision_surface(axs.flat[2], simp_clf, X_train, y_train, title='Simple - Training')
decision_surface(axs.flat[3], simp_clf, X_test, y_test, title='Simple - Test')
fig.tight_layout()

The complex decision tree with default settings partitions the feature space too finely and doesn't generalize well to the test data (i.e. low bias, high variance). The simplified tree generalizes better to the test data.

In [ ]:
# Set up a confusion matrix
from sklearn import metrics
import pandas as pd
import seaborn as sns

def pretty_matrix(y_true, y_pred, names):
    cm = metrics.confusion_matrix(y_true, y_pred)
    cm = pd.DataFrame(cm, index = names, columns= names)
    cm.index.name = "Observed"
    cm.columns.name = "Predicted"
    fig, ax = plt.subplots(figsize=(4,4))
    sns.heatmap(cm, annot=True, cmap="viridis", ax=ax)
    return fig

In [ ]:
# Accuracy of default model
y_pred = def_clf.predict(X_test)
fig = pretty_matrix(y_test, y_pred, iris.target_names)
print(metrics.classification_report(y_test, y_pred, target_names=iris.target_names))
print("Overall Accuracy=", metrics.accuracy_score(y_test, y_pred).round(2))

In [ ]:
# Accuracy of simple model
y_pred = simp_clf.predict(X_test)
fig = pretty_matrix(y_test, y_pred, iris.target_names)
print(metrics.classification_report(y_test, y_pred, target_names=iris.target_names))
print("Overall Accuracy=", metrics.accuracy_score(y_test, y_pred).round(2))

We can see this effect of overfitting when comparing the accuracy statistics as well. The simplied decision tree has higher overall accuracy and yields better precision and recall for the two classes with less support.  

In practice tuning a decision tree to reduce overfitting while maintaining high accuracy can be challenging. Overfitting can also be reduced by "pruning" nodes that don't significantly improve the accuracy of tree after the model has already been trained.

**Pro: Easy to interpret, no data prep needed, use for classification or regression, etc.**  

**Con: Prone to overfitting and often difficult to achieve high accuracies**

# Hence Random Forests...

Random Forests solves the overfitting problem and improves accuracy by aggregating predictions from an ensemble of decision trees which incorporate randomness in two ways:  

*The Randomness in Random Forests*
1. Bagging - Each tree is trained with a random sample of the data that is taken with replacement (usually the size of the original dataset), which causes about 1/3 of the samples to be excluded from each tree. This bootstrapped sampling decorrelates the trees which reduces overall model variance.  


2. Subset of features - Splitting at each node considers a random subset of all the features (usually the square root of the number of features). This also decorrelates trees by preventing dominance of the same features in every tree.

*Aggregating Trees*  
Each tree is typically grown to full depth with no pruning. Model bias is increased and variance is reduced by aggregating multiple trees.  
* Classification - simple or weighted vote counting  
* Regression - simple or weighted mean

<img src="https://www.analyticsvidhya.com/wp-content/uploads/2015/06/random-forest7.png" style="width: 50%; height: 50%">

## Quick Revisit with Iris

Lets see how this method performs on the Iris dataset we previously looked at.

In [ ]:
# A complex decision tree with default settings
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100) # 100 will be the new default soon
rf_clf = rf_clf.fit(X_train, y_train)

rf_clf.get_params()

In [ ]:
# Accuracy of default random forest model
y_pred = rf_clf.predict(X_test)
fig = pretty_matrix(y_test, y_pred, iris.target_names)
print(metrics.classification_report(y_test, y_pred, target_names=iris.target_names))
print("Overall Accuracy=", metrics.accuracy_score(y_test, y_pred).round(2))

Already a small improvement over the previous models. We're only using two features so random selection in node splitting isn't benefitting the model as much in this example as it would for most real world applications.

In [ ]:
# Look at one of the trees
dot_data = tree.export_graphviz(rf_clf.estimators_[0], out_file=None, 
                     feature_names=iris.feature_names[:2],  
                     class_names=iris.target_names,  
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)
graph

This tree becomes different from our previous decision tree after the first split. Since trees are decorrelated they will each be a little different and looking at individual trees may not help much with understanding the overall model. So we lose a lot of the interpretability of a simple decision tree.

## Land cover classification

Iris is great for simple examples, but to get a better sense of how to work with random forest lets use a slightly more complex dataset by doing land cover classification from satellite imagery.

*Landsat 8 - Operational Land Imager*

<img src="https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/styles/atom_page_medium/public/thumbnails/image/Landsat_8_%28LDCM%29_Satellite_over_Earth%2C%20Wiki%20Commons.jpg?itok=8broIIlp">

*Landsat 8 Spectral Response*

<img src="https://github.com/FevetS/ds_random_forest/raw/master/media/l8_spectral_response.PNG">

https://landsat.usgs.gov/spectral-characteristics-viewer

In [ ]:
# Load data and train test split
data = pd.read_csv(r"F:\temp\MN_l8_2013med_samp_filt.csv")

X = data.drop(['landcover'], axis=1)
y = data['landcover']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42)

train = pd.concat([X_train, y_train], axis=1)

In [ ]:
data['landcover'].value_counts()

In [ ]:
# pairplot of original bands colored by class
bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
sns.pairplot(train[bands + ['landcover']], hue='landcover')

Many of the original bands are highly correlated, and they don't seperate the classes very well.

*Spectral Indices*

Normalized Difference Vegetation Index (NDVI) = (NIR - Red)/(NIR + Red)  

Normalized Burn Ratio (NBR) = (NIR - SWIR2)/(NIR + SWIR2)  

Normalized Difference Moisture Index (NDMI) = (NIR - SWIR1)/(NIR + SWIR1)  

Brightness, Greenness, Wetness: Tasseled-Cap Indices, which are basically the first three principal components of the original band values.

In [ ]:
# pairplot of features colored by class
specix = ['ndvi', 'nbr', 'ndmi', 'greenness', 'wetness', 'brightness']
sns.pairplot(train[specix + ['landcover']], hue='landcover')

#### A quick classification

In [ ]:
rf = RandomForestClassifier(n_estimators=100, oob_score=True).fit(X_train, y_train)

y_pred = rf.predict(X_test)
fig = pretty_matrix(y_test, y_pred, np.unique(y_test))
print(metrics.classification_report(y_test, y_pred))
print("Overall Accuracy=", metrics.accuracy_score(y_test, y_pred).round(2))

## Useful Features of Random Forest

### Out-of-Bag Error

Since random forest uses a bootstrapped sample for each tree, the withheld (or out-of-bag) samples can be used for internal cross-validation. Each sample is predicted using the trees which did not include it for training. Then the predicted values are compared against the true values.

The out-of-bag error is usually a good unbiased estimator of accuracy and it can be used for model tuning.

In [ ]:
# Compare OOB score to test score (score=fraction of samples correctly classified, i think)
print("Out-of-bag error =", rf.oob_score_.round(2))
print("Test error = ", rf.score(X_test, y_test).round(2))

In [ ]:
# Class prediction probability for each sample from the OOB prediction,
# which enables calculation of more error stats if needed
pd.DataFrame(rf.oob_decision_function_, index = y_train.index, columns = np.unique(y_train)).head()

### Variable Importance

Random Forest can also be used find the relative importance of each feature. This is sometimes used for feature selection even if another model is ultimately used for prediction.  

Feature importance is often defined as the mean decrease in Gini impurity for the feature across all trees and weighted by the number of the samples that reach the node where the feature is used. Reduction in out-of-bag error is another common method.

In [ ]:
# importances are an attribute of the model object
rf.feature_importances_

In [ ]:
# but you can get the importance from each tree for plotting variability
imps = [tree.feature_importances_ for tree in rf.estimators_]
imps = pd.DataFrame(imps, columns=X_train.columns)
ranked = imps.mean().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(8,3))
sns.violinplot('variable', 'value', data=imps.melt(), order=ranked.index, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=30);

However, many features are highly correlated...

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
sns.heatmap(X_train.corr().round(2), annot=True, cmap="RdBu", ax=ax)

... and yet their relative importance can sometimes be very different.  

Random Forest will simply choose the best feature at a particular node which mean that even a very similar feature may only be used further down the tree where it causes less decrease in impurity. This can sometimes give the false impression that a feature is unimportant when it fact it may be effectively interchangeable with a very important feature. Or, two highly correlated features could have nearly the same importance and all subsequent feature importances get diluted.

## Benefits of Random Forest  
Including all the best parts of decision trees.

### 1. Unaffected by multicollinearity

In [ ]:
# Remove correlated predictors and check accuracy
X_tcor = X_train.drop(['swir2', 'blue', 'green', 'brightness', 'nbr'], axis=1)
rf_nocorr = RandomForestClassifier(n_estimators=100, oob_score=True).fit(X_tcor, y_train)

X_vcor = X_test.drop(['swir2', 'blue', 'green', 'brightness', 'nbr'], axis=1)                                                             
y_pcor = rf_nocorr.predict(X_vcor)
fig = pretty_matrix(y_test, y_pcor, np.unique(y_test))
print(metrics.classification_report(y_test, y_pcor))
print("Overall Accuracy=", metrics.accuracy_score(y_test, y_pcor).round(2))

Overall accuracy may actually drop a little because some explanatory power could be lost even though the dropped predictors were highly correlated with kept predictors. So correlated predictors don't often decrease the model accuracy.

### 2. Ignores uninformative predictors

Uninformative predictors simply don't get used in splitting because they don't decrease impurity, so they have no affect on accuracy.

In [ ]:
# Add useless features and check accuracy 
useless = pd.DataFrame(np.random.rand(len(X_train), 3), index=X_train.index)
X_trand=  pd.concat([X_train, useless], axis=1)
rf_rand = RandomForestClassifier(n_estimators=100, oob_score=True).fit(X_trand, y_train)

useless = pd.DataFrame(np.random.rand(len(X_test), 3), index=X_test.index)
X_vrand = pd.concat([X_test, useless], axis=1)                                                             
y_prand = rf_rand.predict(X_vrand)
print("Overall Accuracy=", metrics.accuracy_score(y_test, y_prand).round(2))
pd.Series(rf_rand.feature_importances_, index=X_trand.columns).sort_values(ascending=False)

All the random predictors (0,1,2) have the lowest feature importance.

### 3. No need to transform or scale the data

In [ ]:
# scale some predictors and check accuruacy and variable importance
X_tscale = X_train.copy()
X_tscale[specix] = X_tscale[specix] * 1e7

rf_scale = RandomForestClassifier(n_estimators=100, oob_score=True).fit(X_tscale, y_train)

X_vscale = X_test.copy()
X_vscale[specix] = X_vscale[specix] * 1e7                                                           
y_pscale = rf_scale.predict(X_vscale)
print("Overall Accuracy=", metrics.accuracy_score(y_test, y_pscale).round(2))
# check variable importance
pd.Series(rf_scale.feature_importances_, index=X_tscale.columns).sort_values(ascending=False)

Accuracy and feature ranking is maintained after scaling some predictors.

### 4. Works with categorical predictors

In [ ]:
# convert one predictor to categorical and encode, then run model
cat = pd.cut(X_train['ndvi'], bins=[-1,0,0.25, 1], labels=['l','m','h'])
encoded = pd.get_dummies(cat, prefix='ndvi')
X_tcat = pd.concat([X_train.drop('ndvi', axis=1), encoded], axis=1)

rf_cat = RandomForestClassifier(n_estimators=100, oob_score=True).fit(X_tcat, y_train)

cat = pd.cut(X_test['ndvi'], bins=[-1,0,0.25, 1], labels=['l','m','h'])
encoded = pd.get_dummies(cat, prefix='ndvi')
X_vcat = pd.concat([X_test.drop('ndvi', axis=1), encoded], axis=1)                                                          
y_pcat = rf_cat.predict(X_vcat)

print("Overall Accuracy=", metrics.accuracy_score(y_test, y_pcat).round(2))
pd.Series(rf_cat.feature_importances_, index=X_tcat.columns).sort_values(ascending=False)

The model doesn't complain about the one-hot-encoded predictors, but they lose some explanatory power and a correlated predictor takes their place in the feature ranking.

### 5. Tuning often not necessary

Random Forest tends to work very well with default settings, but if you want to eak out another 0.5% increase in accuracy you can try tweaking:
1. n_estimators - the number of trees
2. max_features - the number of variables considered at each node
3. any of the decision tree parameters 

In [ ]:
# Create and initiate the parameter grid 
from sklearn.model_selection import GridSearchCV

grid = {
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': [1, 2, 5],
    'n_estimators': [10, 20, 50, 100, 200, 300, 500]
}

grid_search = GridSearchCV(estimator = RandomForestClassifier(),
                           param_grid = grid, cv = 3, n_jobs = 4)

In [ ]:
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

results = pd.DataFrame(grid_search.cv_results_)
results.head()

In [ ]:
sub = results[(results.param_max_features=='sqrt') & (results.param_min_samples_leaf==1)]

fig, ax = plt.subplots(figsize=(5,3))
ax.errorbar(sub['param_n_estimators'], sub['mean_test_score'], yerr=sub['std_test_score'])
ax.set(xlabel='n_estimators', ylabel='mean_test_score');

The change in accuracy is pretty minimal after 100 trees, which is a standard value.


I think the primary benefit of tuning for Random Forest is actually for reducing processing time for large datasets. Reducing the number of trees or the complexity of the trees can save time for training and prediction. 

For example, training time scales linearly with the number of trees. Training takes 5 times as long for 500 trees as it does for 100 trees, but accuracy doesn't improve at all.

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
ax.errorbar(sub['param_n_estimators'], sub['mean_fit_time'], yerr=sub['std_fit_time'])
ax.set(xlabel='n_estimators', ylabel='mean_fit_time');

Alternatively, we can use the out-of-bag errors to evaluate the model in the grid search, so the data doesn't need to be further divided.  
But, we don't easily get the fancy results dictionary provided by GridSearchCV.

In [ ]:
# Loop over the grid and use OOB to get the best params
from sklearn.model_selection import ParameterGrid

rfg = RandomForestClassifier(oob_score=True)
best_score=0
for g in ParameterGrid(grid):
    rfg.set_params(**g)
    rfg.fit(X_train, y_train)
    # save if best
    if rfg.oob_score_ > best_score:
        best_score = rfg.oob_score_
        best_grid = g

print("OOB accuracy=", best_score.round(2))
print("Grid:", best_grid)

### 6. Supposed to work with missing data

CART models can handle missing values by choosing a surrogate split, which means Random Forest should have the same ability. Unfortunately, I can't find an implementation which uses surrogate splits. But you might be able to roll your own with [rpart](https://cran.r-project.org/web/packages/rpart/rpart.pdf).

### 7. Robust to overfitting
As evidenced by the OOB accuracy roughly matching the test set accuracy.

# Drawbacks

### 1. Lost interpretability  
Having multiple trees grown to full depth makes the model far too difficult to interpret, but we can get some understanding of the relative influence of the predictors by looking at variable importance (accounting for multicollinearity) and through partial dependence plots. 

Partial dependence plots show the marginal effect of a feature on the model prediction. The partial dependence function gives the average prediction for a particular value of our feature of interest by iterating over all other feature values through a  a Monte-Carlo simulation (I think). Unfortunately, the PDP will show some unrealistic values if the other features are correlated with our feature of interest, which is clearly a problem we have with this dataset.

See this site for a much better explanation: https://christophm.github.io/interpretable-ml-book/pdp.html

Scikit-learn's partial dependence functions can only be used with Gradient Boosting, but PDPbox works with Random Forest. Skater is another package for model interpretation. Also see iml or pdp in R.

In [ ]:
#!pip install pdpbox
from pdpbox import pdp, info_plots

In [ ]:
# create grid used to make the plot (monte-carlo sim?)
pdp_ndvi = pdp.pdp_isolate(
    model=rf, dataset=train, model_features=X_train.columns, feature='ndvi'
)

# make the PD plot
fig, axes = pdp.pdp_plot(
    pdp_isolate_out=pdp_ndvi, feature_name='ndvi', center=True,
    ncols=3, plot_lines=True, plot_pts_dist=True, frac_to_plot=100
)

# set class names as x-labels
for i, ax_dict in enumerate(axes['pdp_ax']):
    ax = ax_dict['_count_ax']
    ax.set(xlabel = "ndvi (" + rf.classes_[i] + ")",
           title = '')

PDPbox's dependence plots show the average probablity of prediction of a class across the range of values for a particular feature (black/yellow line). It also shows the Individual Conditional Expectation (ICE plot), which has the marginal effect of the feature for all data points (thin blue lines). The lines below the x-axis is a rug plot of data points, so there is greater uncertainty in the prediction probability where there are fewer data points.

Remember that NDVI is an indicator of vegetation quantity and health, thus as NDVI increases the probability of predicting a vegetation class (forest, herbaceous, or shrubs) tends to increase.

In [ ]:
# calculate pdp for interaction of two features
pdp_ndvi_ndmi = pdp.pdp_interact(
    model=rf, dataset=train, model_features=X_train.columns, features=['ndvi', 'ndmi'], 
    num_grid_points=[10, 10],  percentile_ranges=[None, None], n_jobs=4
)

In [ ]:
# make the pdp interact plots
fig, axes = pdp.pdp_interact_plot(
    pdp_ndvi_ndmi, ['ndvi', 'ndmi'], plot_type='grid', ncols=2, plot_pdp=True, which_classes=[6, 7]
)

The interact plot shows a heatmap of the prediction probability across the distribuation of two marginal features (ndvi and ndmi). We can see that class 6 (water) has a high prediction probability when ndmi is very high and ndvi is very low. Class 7 (wetlands) tend to be predicted when ndmi and ndvi are both moderate.

### 2. Susceptible to imbalanced data  
Like many other models, Random Forest will tend to favor an overly dominant class. You can counteract this by either setting **class weight** or through **sampling**. The dominant class can be *undersampled* or the rare classes can be *oversampled* to improve class balance. There are a variety of sampling methods for handling imbalanced data.

### 3. Usually not the best model

Random Forest is great for its versatility, ease of use, and reasonable performance, which is why many people reach for it first.  
However, if you're willing to put the time and effort into building a more complex model then you'll likely get better accuracy.  